In [110]:
import sys
import os
import pandas as pd
import sqlite3
import ebisu
from memorizesrs import schedule
from datetime import datetime, timedelta
from math import inf


COLLECTION_PATH = "/Users/jacob/Library/Application Support/Anki2/Minerva/collection.anki2"
cnx = sqlite3.connect(COLLECTION_PATH)

# most commonly reviewed card
df = pd.read_sql_query("SELECT * FROM revlog WHERE cid = 1286321546300", cnx)

# currentcardid = df.cid.value_counts().idxmax()

In [111]:
learnTime = datetime.fromtimestamp(1286321546300 / 1000.00)

reviewTimes = pd.to_datetime(df['id'], unit = "ms")

reviewResults = df['ease'] != 1 

initialModel = ebisu.defaultModel(0.25, 3) # initial half life is 15 minutes, alpha = beta = 3

In [112]:
def historyToEbisuModel(learnTime, reviewTimes, reviewResults, initialModel = ebisu.defaultModel(0.25,3)):
    """Convert history of quizzes to Ebisu model"""
    
    assert len(reviewTimes) == len(reviewResults)

    previousTime = learnTime
    model = initialModel
    
    for (reviewTime, result) in zip(reviewTimes, reviewResults):
        
        model = ebisu.updateRecall(model, result, (reviewTime - previousTime).total_seconds() / 3600)
        previousTime = reviewTime
        
    return model


ebisuModel = historyToEbisuModel(learnTime, reviewTimes, reviewResults)

ebisuModel

(10.813347396170563, 11.797213496966025, 144.61125755257288)

In [117]:
def ebisuModelToMemorizeInterval(model, q=10.0, T=inf):
    """Use an Ebisu model to draw a MEMORIZE interval"""
    return schedule(lambda t: ebisu.predictRecall(initialModel, t, exact=True), q, T)

dueIn = ebisuModelToMemorizeInterval(ebisuModel, q=10.0, T=inf)
print(f"Quiz due in {dueIn} hours")


Quiz due in 2.605206069833358 hours


In [118]:
for model in [initialModel, ebisuModel]:
    print("Ebisu halflife={} hours".format(ebisu.modelToPercentileDecay(model)))
    print(pd.DataFrame([ebisuModelToMemorizeInterval(model) for n in range(10_000)]).describe())

Ebisu halflife=0.25000000000000017 hours
                  0
count  10000.000000
mean       3.613281
std        3.203966
min        0.016297
25%        1.346231
50%        2.722674
75%        4.882248
max       32.055608
Ebisu halflife=135.61910906105547 hours
                  0
count  10000.000000
mean       3.530936
std        3.139008
min        0.007428
25%        1.315331
50%        2.632318
75%        4.763936
max       36.387621
